# Fully Dynamic k-center Clustering

## Parsing

This section has to be runned only once; its role is to parse the data and create a .hdf5 dataset with three fields: one for the timestamp, one for the latitude, one for the longitude. We will use this file in the following sections.

In [1]:
import h5py
def parse_input(path):
    dataset = h5py.File("dataset.hdf5", "w")
    timestamps = dataset.create_dataset("timestamps", (1000001,), dtype='i')
    latitudes = dataset.create_dataset("latitudes", (1000001,), dtype='f')
    longitudes = dataset.create_dataset("longitudes", (1000001,), dtype='f')
    f = open(path,'r')
    for i, line in enumerate(f):
        line = line.split()
        timestamps[i] = int(line[0])
        latitudes[i] = float(line[1])
        longitudes[i] = float(line[2])
    dataset.close()

In [2]:
parse_input('dataset/twitter_1000000.txt')

In [ ]:
import h5py
f = h5py.File("dataset.hdf5", "r")

In [11]:
timestamps = f['timestamps']
latitudes = f['latitudes']
longitudes = f['longitudes']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
dataset = list(zip(latitudes, longitudes))
sample_dataset = random.sample(dataset, 2000)
sample_latitudes = [sample[0] for sample in sample_dataset]
sample_longitudes = [sample[1] for sample in sample_dataset]
print(sample_dataset[10], sample_latitudes[10], sample_longitudes[10])
#plt.scatter(,)
plt.show()


In [26]:
from scipy.spatial.distance import pdist
# to compute pairwise distances

In [15]:
import random
l = [[1, 2, 3], [4,  5, 6], [7, 8, 9], [45, 5, 7]]
random.choice(l)

[45, 5, 7]